In [1]:
import numpy as np

# Built-in Numpy Ufuncs

* `unary ufunc` ve `binary ufunc`

* `abs, exp, sqrt, add, max ...`

## unary ufunc

****take one array (ndarray) as the argument****

In [1]:
import numpy as np

def my_add_two(x):
    return x + 2

#create ufunc
np_my_add_two = np.frompyfunc(my_add_two, 1, 1)
# check types
print(type(my_add_two), type(np_my_add_two))

<class 'function'> <class 'numpy.ufunc'>


* Unutmayalım! İkiside aynı şey işlemi yapsa da **np** bunu vectorize bir şekilde yapar ve zamandan(hatta belki memoryden) tasarruf eder.

In [2]:

def my_sum(x, y):
    return x + y


#create ufunc
np_my_sum = np.frompyfunc(my_sum, 2, 1)
# check types
print(type(my_sum), type(np_my_sum))

<class 'function'> <class 'numpy.ufunc'>


## Numba ufunc creation: the vectorize decorator

* Numba'nın `@vectorize` dekoratörü ile oluşturulan bir fonksiyon, derleme sayesinde C ile yazılmış NumPy ufunc'ları kadar hızlı çalışabilir

In [3]:
from numba import vectorize, int64, float32, float64


# create default ufunc with datatypes conversion
@vectorize(
    [
    int64(int64,int64), 
    float32(float32,float32), 
    float64(float64,float64)
    ],
    nopython=True,
    cache=True,
    fastmath=True,
)
def numba_dtype_opt_sum(x, y):
    return x + y

# check type
print(type(numba_dtype_opt_sum))

# check that function works on scalars
print(numba_dtype_opt_sum(1, 1))

# check that function works on arrays
print(numba_dtype_opt_sum(np.ones(2), np.ones(2)))

<class 'numba.np.ufunc.dufunc.DUFunc'>
2
[2. 2.]


## ufunc creation: NumPy vs. Numba

**In this section, we will compare the computation times from several approaches to add two 2D NumPy arrays (ndarray). We will use the following ufuncs:**

* NumPy **built-in** add `(np.add)`

* `np_my_sum`, **ufunc created with** `np.frompyfunc`

* `numba_lazy_sum`, **ufunc created with Numba’s vectorize decorator**, with default arguments and no dtypes information

* `numba_dtype_sum`, **ufunc created with the vectorize decorator**, *stating dtype conversion* for compilation

* `numba_dtype_opt_sum`, **ufunc created with the vectorize decorator**, stating dtype conversion for compilation *plus optimizing arguments* for performance

In [ ]:
from time import time
# create test 2D arrays 10_000 x 10_000
l = 10_000
a = np.arange(l * l).reshape(l, l)
b = a.copy()


# NumPy built-in binary add
start = time()
print(np.add(a, b))
end = time()
print("numpy built-in binary add:", end - start)

# NumPy ufunc created from my_sum (ordinary Python function)
def my_sum(x, y):
    return x + y

start = time()
np_my_sum = np.frompyfunc(my_sum, 2, 1)
print(np_my_sum(a, b))
end = time()
print("numpy ufunc frommyfunc", end - start)

# Numba ufunc lazy mode from my_sum (ordinary Python function)
@vectorize 
def numba_lazy_sum(x, y):
    return x + y
start = time()
print(numba_lazy_sum(a, b))
end = time()
print("numba ufunc lazy mode:", end - start)

# Numba ufunc with dtypes from my_sum (ordinary Python function)
@vectorize(
    [
    int64(int64,int64), 
    float32(float32,float32), 
    float64(float64,float64)
    ]
)
def numba_dtype_sum(x, y):
    return x + y
start = time()
print(numba_dtype_sum(a, b))
end = time()
print("numba ufunc with dtypes:", end - start)

# Numba ufunc with dtypes and other 
# fancy vectorize arguments from sum (ordinary Python function)
@vectorize(
    [
    int64(int64,int64), 
    float32(float32,float32), 
    float64(float64,float64)
    ],
    nopython=True,
    cache=True,
    fastmath=True,
)
def numba_dtype_opt_sum(x, y):
    return x + y
start = time()
print(numba_dtype_opt_sum(a, b))
end = time()
print("numba ufunc with dtypes and other vectorize arguments:", end - start)

[[        0         2         4 ...     19994     19996     19998]
 [    20000     20002     20004 ...     39994     39996     39998]
 [    40000     40002     40004 ...     59994     59996     59998]
 ...
 [199940000 199940002 199940004 ... 199959994 199959996 199959998]
 [199960000 199960002 199960004 ... 199979994 199979996 199979998]
 [199980000 199980002 199980004 ... 199999994 199999996 199999998]]
numpy built-in binary add: 0.20944690704345703
[[0 2 4 ... 19994 19996 19998]
 [20000 20002 20004 ... 39994 39996 39998]
 [40000 40002 40004 ... 59994 59996 59998]
 ...
 [199940000 199940002 199940004 ... 199959994 199959996 199959998]
 [199960000 199960002 199960004 ... 199979994 199979996 199979998]
 [199980000 199980002 199980004 ... 199999994 199999996 199999998]]
numpy ufunc from ordinary Python function: 11.20490312576294
[[        0         2         4 ...     19994     19996     19998]
 [    20000     20002     20004 ...     39994     39996     39998]
 [    40000     40002     

* Bu 4 ünün süreleri zaten çok yakın olmalı. Ama `Numpy ufunc frommyfunc` un süresi bunlarda oldukça fazla.

# Custom Ufuncs

* `np.fromyufunc()`